In [ ]:
import pandas as pd
import numpy as np
import json
from collections import Counter

## Data Processing

In [ ]:
df = pd.read_csv("job_descriptions.csv")

In [ ]:
df = pd.read_csv("job_descriptions.csv")

In [ ]:
feature_to_filter_type = "Work Type"
values_to_keep_type = ["Full-Time"]

df_filtered_type = df[df[feature_to_filter_type].isin(values_to_keep_type)]


In [ ]:
feature_to_filter_country = "Country"
values_to_keep_country = ["USA"]

df_filtered_country = df[df[feature_to_filter_country].isin(values_to_keep_country)]


In [ ]:
import json

def extract_sector_industry(row):
  try:
    profile = json.loads(row['Company Profile'])
    return profile.get('Sector'), profile.get('Industry')
  except:
    return None, None

df_filtered_country[['Sector', 'Industry']] = df_filtered_country.apply(extract_sector_industry, axis=1, result_type='expand')


In [ ]:
df_filtered_country.drop(columns=['Company Profile'], inplace=True)

In [ ]:
df_filtered_country.drop(columns=['Job Id']).describe()

## Sector & Industry Prediction

In [ ]:
import requests
import json

url = "http://localhost:11434/api/chat"

In [ ]:
def llama3(prompt):
    data = {
        "model": "llama3",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False,
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()["message"]["content"]

In [ ]:
sectors_list = ['Aerospace Building Systems', 'Aerospace and Defense', 'Agriculture', 
                'Airlines', 'Appliances and Electronics', 'Automotive', 
                'Automotive Motorcycles', 'Automotive Parts', 'Aviation and Travel', 
                'Banking', 'Beauty', 'Beverage Alcohol', 'Beverages', 'Business Services', 
                'Chemicals', 'Communication Equipment', 'Conglomerate', 
                'Construction Building Materials', 'Consumer Goods', 'Diversified', 
                'Electrical Equipment', 'Electronics', 'Energy', 'Energy Oil and Gas', 
                'Engineering', 'Entertainment', 'Equipment Rental', 'Financial Services', 
                'Financial Services Investment Management', 'Financial Technology', 
                'Financials', 'Flavor and Fragrances', 'Food and Beverage', 'Healthcare', 
                'Healthcare Services', 'Hospitality', 'Hospitality Hotels', 'IT Consulting', 
                'IT Services', 'Industrial Conglomerates', 'Industrial Machinery', 
                'Industrial Supplies', 'Information Technology', 'Infrastructure', 
                'Insurance', 'Lighting and Technology', 'Logistics', 'Lubricants', 
                'Manufacturing', 'Media', 'Media Entertainment', 'Medical Devices', 
                'Medical Technology', 'Metals', 'Metals and Mining', 'Mining and Metals Gold', 
                'Motion Control', 'Natural Gas', 'Oil and Gas', 'Oil and Gas Services', 
                'Packaging', 'Payroll   HR Services', 'Pharmaceuticals', 'Postal Services', 
                'Real Estate', 'Real Estate Construction', 'Restaurants', 'Retail', 
                'Retail and Conglomerate', 'Services', 'Shipping and Logistics', 
                'Software and Technology', 'Technology', 'Technology IT Services', 
                'Technology and Networking', 'Technology and Search', 
                'Technology and Software', 'Telecommunications', 'Timber   Paper', 'Tobacco', 
                'Transportation', 'Travel and Booking', 'Utilities']


In [ ]:
industry_list = ['Real Estate' 'Aerospace and Defense' 'Chemicals' 'Commercial Banks'
 'Utilities: Gas and Electric' 'Pharmaceuticals' 'Airlines' 'Beverages'
 'Petroleum Refining' 'Banking & Financial Services' 'Entertainment'
 'Internet Services and Retailing' 'Real Estate/REIT'
 'Information Technology' 'Semiconductors and Other Electronic Components'
 'Financial Data Services' 'Food Services' 'Automotive'
 'Electrical Equipment' 'Travel and Leisure - Hotels'
 'Energy - Oil & Gas Exploration & Production'
 'Mining, Crude-Oil Production' 'Financial Services'
 'Wholesalers: Diversified' 'Financial Services - Investment'
 'Transportation/Logistics' 'Wholesalers: Health Care' 'Mining'
 'Automotive Retailing, Services' 'Telecommunications'
 'Diversified Financials' 'Packaging, Containers' 'Industrial Machinery'
 'Business Services - Pest Control' 'Technology & Telecommunications'
 'Aerospace & Defense' 'Food and Drug Stores' 'Food Production'
 'Pipelines' 'Consumer Goods' 'Specialty Retailers: Other'
 'Home Equipment, Furnishings' 'Tobacco' 'Health Care: Medical Facilities'
 'Energy' 'Industrial Conglomerate' 'Electronics & Appliances'
 'General Merchandisers' 'Diversified Outsourcing Services' 'Oil and Gas'
 'Equipment Rental' 'Airlines & Aviation' 'Media and Entertainment'
 'Banking' 'Insurance: Life, Health (Stock)' 'Food & Beverage'
 'Forest and Paper Products' 'Banking/Financial Services'
 'Postal Services' 'Insurance - Property/Casualty'
 'Insurance: Property and Casualty (Stock)'
 'Food and Beverage - Alcoholic Beverages' 'Energy - Oil & Gas Services'
 'Metals' 'Food Consumer Products' 'Retail' 'Transportation and Logistics'
 'Medical Products and Equipment' 'Information Technology Services'
 'Computer Software' 'Engineering & Construction' 'Utilities'
 'Health Care: Insurance and Managed Care' 'Securities'
 'Paints and Coatings' 'Gas and Energy' 'Trucking, Truck Leasing'
 'Healthcare' 'Restaurants' 'Insurance' 'Technology' 'Internet/Technology'
 'Building Materials/Construction' 'Electronics/Components'
 'Financial Services/Insurance' 'Software' 'Lighting & Electronics'
 'Network and Other Communications Equipment'
 'Technology & Search Engines' 'Wholesalers: Food and Grocery'
 'Publishing, Printing' 'Household and Personal Products'
 'Engineering/Industrial Equipment' 'Food and Beverage'
 'Insurance: Life, Health (Mutual)' 'Motor Vehicles & Parts' 'Oil & Gas'
 'Health Care: Pharmacy and Other Services' 'Retail - Books & News'
 'Real Estate/Construction' 'Food Manufacturing'
 'Hotels, Casinos, Resorts' 'Retail - Farm and Ranch Supplies'
 'Diversified' 'Cement and Building Materials'
 'Logistics & Delivery Services' 'Banking and Financial Services'
 'Wholesalers: Electronics and Office Equipment'
 'Manufacturing - Diversified' 'Energy - Diversified'
 'Building Materials, Glass' 'Mail, Package and Freight Delivery'
 'Hospitality & Entertainment' 'Conglomerate' 'Gambling/Gaming'
 'Computers, Office Equipment' 'Travel and Leisure - Cruises'
 'Temporary Help' 'Financial Services/Investment Management'
 'Apparel and Textiles' 'Homebuilders' 'Steel'
 'Technology & Entertainment' 'Personal Care & Cosmetics' 'Apparel'
 'Manufacturing/Building Systems' 'Distribution/Wholesale'
 'Transportation - Rail' 'Technology & Social Media' 'Energy - Utilities'
 'Consumer Products - Cleaning Products' 'Advertising, Marketing'
 'Waste Management' 'E-commerce & Technology' 'Construction/Engineering'
 'Railroads' 'Financial Services - Investment Banking'
 'Retail - Food & Drug' 'Specialty Retailers: Apparel'
 'Industrial Manufacturing' 'Cement and Concrete' 'Tire Manufacturing'
 'Insurance: Property and Casualty (Mutual)' 'Beverage'
 'Construction and Farm Machinery'
 'Scientific, Photographic and Control Equipment'
 'Advertising and Marketing' 'Manufacturing - Electrical Equipment'
 'Financial Technology (Fintech)' 'Transportation - Airlines'
 'Information Technology and Services' 'Electronics, Electrical Equip.'
 'Transportation/Infrastructure' 'Electronics/Instrumentation'
 'Apparel/Fashion' 'Agrochemicals' 'Hospitality/Hotels'
 'E-commerce/Technology' 'Retail - Discount/Department Stores' 'Logistics'
 'Manufacturing & Transportation' 'Retail - General Merchandise'
 'Retail - Home Improvement' 'Transportation' 'Medical Devices/Healthcare'
 'Transportation Equipment' 'Ports and Infrastructure'
 'Manufacturing/Steel' 'Entertainment - Streaming Services'
 'Business Services - Testing/Compliance' 'Semiconductors'
 'Technology & Electronics' 'Consumer Products - Cosmetics'
 'Online Recruitment' 'Entertainment - Satellite Radio' 'Retail - Apparel'
 'Construction' 'Retail/Home Improvement' 'Jewelry and Watches'
 'Renewable Energy' 'Gaming/Gambling' 'Software/Technology'
 'Oil and Gas Equipment, Services' 'Gambling'
 'Business Services - Data/Analytics' 'Energy - Coal' 'Aviation/Airlines'
 'Retail - Electronics' 'Technology - Ride-hailing'
 'Construction & Building Materials' 'Education' 'Life Sciences'
 'Cement and Chemicals' 'Entertainment - Music Streaming' 'E-commerce'
 'Construction and Engineering' 'Media & Entertainment' 'Electronics'
 'Equipment Leasing' 'Technology & Security'
 'Aluminum and Copper Manufacturing' 'Apparel & Footwear'
 'Cloud Software and Services' 'Media/Entertainment']


In [ ]:
def getSector(prompt):
    sectors_str = ", ".join(sectors_list)
    industry_str = ", ".join(industry_list)
    data = {
        "model": "llama3",
        "messages": [
            {
                "role": "user",
                # What we are asking the API, will be given the name of company in the prompt
                "content": f"Identify the sector and industry for the company named {prompt}. " 
                            "Respond strictly in the format: 'Sector: [best matching sector], Industry: [best matching industry]'. "
                            f"Select the most appropriate sector from this list: {sectors_str}, and the most appropriate industry from this list: {industry_str}." 
                            "Do not include any additional information or commentary."
            }
        ],
        "stream": False,
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=data)
    
    # extract sector and industry
    try:
        sector = response_content.split("Sector: ")[1].split(",")[0]
        industry = response_content.split("Industry: ")[1].split("]")[0].strip()
    except IndexError:
        sector, industry = "Unknown", "Unknown"

    return sector, industry


In [ ]:
response = getSector("Macquarie Group")
print(response)

In [ ]:
companies = df_filtered_country['company']

In [ ]:
predicted_sectors = []
predicted_industries = []
for company in companies:
    predicted_sector, predicted_industry = getSector(company)
    predicted_sectors.append(predicted_sector)
    predicted_industries.append(predicted_industry)

In [ ]:
df_comparison = pd.DataFrame({
    'company': df_filtered_country['company'],
    'predicted_sector': predicted_sectors,
    'sector': df_filtered_country['sector'],
    'predicted_industry': predicted_industries,
    'industry': df_filtered_country['industry']
})

In [ ]:
df_comparison['sector_correct'] = df_comparison['predicted_sector'] == df_comparison['sector']
df_comparison['industry_correct'] = df_comparison['predicted_industry'] == df_comparison['industry']


In [ ]:
sector_accuracy = df_comparison['sector_correct'].mean()
industry_accuracy = df_comparison['industry_correct'].mean()
